In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 80
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)

    def forward(self, input_, loss_fn):
        one_hot_x = F.one_hot(input_,num_classes=257).float()
        one_hot_x.requires_grad = True
        one_hot_x.retain_grad()
        batch_acc = []
        batch_grad = []
        for _ in range(13):
            x = one_hot_x @ self.embedding.weight
            x = torch.transpose(x, -1, -2)

            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.cat((torch.ones(len(x)).reshape(-1,1),torch.zeros(len(x)).reshape(-1,1)),dim=1).cuda()
            loss = loss_fn(x,mislead_labels)
            loss.backward()
            
            padding_place_mask = (input_ == 0)
            shape = padding_place_mask.shape
            expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)
            grad_min_idx = torch.argmin(one_hot_x.grad,dim=-1)
            grad_sez_mask = (one_hot_x.grad <= 0)
            grad_mask = (expand_padding_place_mask & grad_sez_mask)
            batch_grad.append(torch.div(
                input=(expand_padding_place_mask.float()*one_hot_x.grad).sum(),
                other=expand_padding_place_mask.float().sum()
            ).detach().cpu().numpy())
            new_x = torch.argmin(grad_mask.float()*one_hot_x.grad,dim=-1)
            grad_all_gz_mask = grad_sez_mask.sum(dim=-1) == 257
            mask = ((~grad_all_gz_mask) & padding_place_mask)
            temp = (1 - mask.float())*input_ + mask.float()*new_x
            one_hot_x.data = F.one_hot(temp.long(),num_classes=257).float()
            
            # one_hot_x.grad.zero_()
            
        x = one_hot_x @ self.embedding.weight
        x = torch.transpose(x, -1, -2)

        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()

        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x.detach().cpu().numpy(),batch_acc,batch_grad

In [10]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [11]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [12]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(testloader)
# bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp_label = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp_label[idx,target] = 1
    temp_label = temp_label.cuda() if CUDA else temp_label

#     pred,input_,one_hot_x = model(batch_data,ce_loss)
#     break
    pred,batch_acc,batch_grad = model(batch_data,ce_loss)
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    print(total_batch_acc_str)
    print(total_batch_grad_str)
    print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

  0%|          | 1/1250 [00:05<1:46:43,  5.13s/it]

[0.7875000238 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0250000004 0.0125000002 0.0125000002 0.0125000002 0.0125000002 0.0125000002 0.0125000002 0.0125000002 0.0125000000]
[0.0000000000 -0.0000000270 -0.0000000269 -0.0000000268 -0.0000000267 -0.0000000266 -0.0000000265 -0.0000000264 -0.0000000263 -0.0000000262 -0.0000000261 -0.0000000260 -0.0000000259 -0.0000000258]
test：0.012500, test mean: 0.012500


  0%|          | 2/1250 [00:07<1:10:38,  3.40s/it]

[0.8375000060 0.0250000004 0.0125000002 0.0125000002 0.0125000002 0.0125000002 0.0062500001 0.0062500001 0.0062500001 0.0062500001 0.0062500001 0.0062500001 0.0062500001 0.0062500000]
[0.0000000000 -0.0000000257 -0.0000000256 -0.0000000256 -0.0000000255 -0.0000000255 -0.0000000255 -0.0000000254 -0.0000000254 -0.0000000253 -0.0000000253 -0.0000000253 -0.0000000252 -0.0000000252]
test：0.000000, test mean: 0.006250


  0%|          | 3/1250 [00:08<53:09,  2.56s/it]  

...

[0.8294270949 0.0204627408 0.0133613785 0.0121294073 0.0118990387 0.0117588143 0.0117187502 0.0116686701 0.0116386220 0.0116085739 0.0116085739 0.0115985579 0.0115785259 0.0115484776]
[0.0000000000 -0.0000000289 -0.0000000286 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285]
test：0.012500, test mean: 0.011548


100%|█████████▉| 1249/1250 [30:19<00:01,  1.45s/it]

[0.8293334784 0.0204563655 0.0133606888 0.0121297040 0.0118995198 0.0117594077 0.0117193757 0.0116693357 0.0116393117 0.0116092876 0.0115992796 0.0115892716 0.0115692556 0.0115392314]
[0.0000000000 -0.0000000289 -0.0000000286 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285]
test：0.000000, test mean: 0.011539


100%|██████████| 1250/1250 [30:21<00:00,  1.46s/it]

[0.8292800116 0.0204700004 0.0133600003 0.0121200002 0.0118900002 0.0117500002 0.0117100002 0.0116600002 0.0116300002 0.0116000002 0.0115900002 0.0115800002 0.0115600002 0.0115300000]
[0.0000000000 -0.0000000289 -0.0000000286 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285 -0.0000000285]
test：0.000000, test mean: 0.011530


In [14]:
# import pickle

In [16]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/my_way/cumulative_gradients_v2_13iter_grad.txt','wb') as fp:
#     pickle.dump(total_batch_grad,fp)